In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
# Named Entity Recognition (NER) is a natural language processing task that involves 
# identifying and classifying named entities 
# (such as person names, locations, organizations, dates, etc.) in text.

In [2]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [3]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302


In [4]:
corpus = read_file('train.txt')
print(corpus[0])
print(corpus[1])

[('1', '한편', 'NNG', 'O'), ('1', ',', 'SP', 'O'), ('2', 'AFC', 'SL', 'O'), ('2', '챔피언스', 'NNG', 'O'), ('2', '리그', 'NNG', 'O'), ('3', 'E', 'SL', 'B_OG'), ('3', '조', 'NNG', 'I'), ('3', '에', 'JKB', 'O'), ('4', '속하', 'VV', 'O'), ('4', 'ㄴ', 'ETM', 'O'), ('5', '포항', 'NNP', 'O'), ('6', '역시', 'MAJ', 'O'), ('7', '대회', 'NNG', 'O'), ('8', '8강', 'NNG', 'O'), ('9', '진출', 'NNG', 'O'), ('9', '이', 'JKS', 'O'), ('10', '불투명', 'NNG', 'O'), ('10', '하', 'VV', 'O'), ('10', '다', 'EC', 'O'), ('11', '.', 'SF', 'O')]
[('1', '2003', 'SN', 'B_DT'), ('1', '년', 'NNB', 'I'), ('2', '6', 'SN', 'I'), ('2', '월', 'NNB', 'I'), ('3', '14', 'SN', 'I'), ('3', '일', 'NNB', 'I'), ('4', '사직', 'NNG', 'O'), ('5', '두산', 'NNP', 'O'), ('5', '전', 'NNG', 'O'), ('6', '이후', 'NNG', 'O'), ('7', '박명환', 'NNP', 'B_PS'), ('7', '에게', 'JKB', 'O'), ('8', '당하', 'VV', 'O'), ('8', '았', 'EP', 'O'), ('8', '던', 'ETM', 'O'), ('9', '10', 'SN', 'O'), ('9', '연패', 'NNG', 'O'), ('10', '사슬', 'NNG', 'O'), ('10', '을', 'JKO', 'O'), ('11', '거의', 'MAG', 'O'), ('12'

In [5]:
print(sentences[0:3])

[['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.'], ['2003', '년', '6', '월', '14', '일', '사직', '두산', '전', '이후', '박명환', '에게', '당하', '았', '던', '10', '연패', '사슬', '을', '거의', '5', '년', '만', '에', '끊', '는', '의미', '있', '는', '승리', '이', '었', '다', '.'], ['AP', '통신', '은', '8', '일', '(', '이하', '한국', '시간', ')', '올라주원', ',', '유잉', '을', '비롯', '하', '아', '애드리언', '댄틀리', ',', '팻', '라일리', '감독', ',', '캐시', '러시', '감독', ',', 'TV', '해설가', '딕', '바이텔', ',', '디트로이트', '피스톤스', '의', '구단주', '윌리엄', '데이비드슨', '등', '이', '2008', '명예', '의', '전당', '헌액', '자', '로', '결정', '되', '었', '다고', '보', '아도', '하', '았', '다', '.']]


In [6]:
print(tags[0:3])

[['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B_DT', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'B_PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_DT', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B_OG', 'I', 'O', 'B_DT', 'I', 'O', 'O', 'B_LC', 'O', 'O', 'B_PS', 'O', 'B_PS', 'O', 'O', 'O', 'O', 'B_PS', 'I', 'O', 'B_PS', 'I', 'O', 'O', 'B_PS', 'I', 'O', 'O', 'O', 'O', 'B_PS', 'I', 'O', 'B_OG', 'I', 'O', 'O', 'B_PS', 'I', 'O', 'O', 'B_DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [7]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫 번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower= False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

In [8]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 8
단어 사전 크기 : 13834


In [30]:
print(sent_tokenizer.word_index[0:3])

TypeError: unhashable type: 'slice'

In [10]:
print(len(sent_tokenizer.word_index))

13833


In [11]:
print(tag_tokenizer.word_index)

{'O': 1, 'I': 2, 'B_OG': 3, 'B_PS': 4, 'B_DT': 5, 'B_LC': 6, 'B_TI': 7}


In [12]:
sentences[0]

['한편',
 ',',
 'AFC',
 '챔피언스',
 '리그',
 'E',
 '조',
 '에',
 '속하',
 'ㄴ',
 '포항',
 '역시',
 '대회',
 '8강',
 '진출',
 '이',
 '불투명',
 '하',
 '다',
 '.']

In [13]:
tags[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'B_OG',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [14]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0], len(x_train[0]))  # x_train[0] is the word_indexes of sentences[0] above.
print(y_train[0], len(y_train[0]))  # y_train[0] is the word_indexes of tags[0] above.

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3] 20
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 20


In [15]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

# Named Entity Recognition (NER) is a natural language processing task that involves 
# identifying and classifying named entities 
# (such as person names, locations, organizations, dates, etc.) in text.

In [34]:
# print(index_to_word)  # only opposite order.   word_index <---> index_to_word

In [17]:
print(index_to_ner)

{1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI', 0: 'PAD'}


In [18]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)
print(x_train)
print('---------see elements of x_train---------')
print(x_train[0]) # padding at the end with 0
print(x_train[1])
print('-----------------------------------------')
print(y_train)

[[  183    11  4276 ...     0     0     0]
 [ 1910    42    73 ...     0     0     0]
 [ 6352    11  6353 ...    14     8     3]
 ...
 [  387  1820    13 ...     0     0     0]
 [  531    16 13829 ...     0     0     0]
 [13831   398   451 ...     3     0     0]]
---------see elements of x_train---------
[ 183   11 4276  884  162  931  402   10 2608    7 1516  608  145 1361
  414    4 6347    2    8    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[1910   42   73   44  231   24 2609  430   64  206 4277   84  513   14
   62   67  748 6348    5 1517   55   42   46   10 3252    6  932   23
    6  275    4   18    8    3    0    0    0    0    0    0]
-----------------------------------------
[[1 1 1 ... 0 0 0]
 [5 2 2 ... 0 0 0]
 [2 1 4 ... 1 1 1]
 ...
 [3 2 1 ... 0 0 0]
 [3 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


In [19]:
# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 8)


In [20]:
x_train[0]

array([ 430, 4417,   16,  416,   17,    9, 6582,   13, 6583,    2,   14,
          8,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [21]:
y_train[0] # one-hot encoded!

array([[0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 

In [22]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))

model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
# in the case of LSTM(200), it means that the LSTM layer consists of 200 memory units or hidden units, 
# and each unit performs computations in parallel at each time step. Each memory unit captures and processes information over time, 
# and the outputs of these units are combined and passed on to the next layer or time step.

model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
#TimeDistributed layer: This layer applies a Dense layer to each time step's input separately. 
#It allows the model to learn and apply a prediction for each token in the input sequence.

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])

Epoch 1/10
23/23 [==============================] - 26s 574ms/step - loss: 0.6940 - accuracy: 0.8384
Epoch 2/10
23/23 [==============================] - 12s 527ms/step - loss: 0.3153 - accuracy: 0.9000
Epoch 3/10
23/23 [==============================] - 14s 587ms/step - loss: 0.2140 - accuracy: 0.9275
Epoch 4/10
23/23 [==============================] - 13s 585ms/step - loss: 0.1520 - accuracy: 0.9503
Epoch 5/10
23/23 [==============================] - 13s 546ms/step - loss: 0.1077 - accuracy: 0.9676
Epoch 6/10
23/23 [==============================] - 13s 564ms/step - loss: 0.0759 - accuracy: 0.9774
Epoch 7/10
23/23 [==============================] - 14s 595ms/step - loss: 0.0575 - accuracy: 0.9821
Epoch 8/10
23/23 [==============================] - 13s 579ms/step - loss: 0.0463 - accuracy: 0.9856
Epoch 9/10
23/23 [==============================] - 13s 539ms/step - loss: 0.0378 - accuracy: 0.9882
Epoch 10/10
23/23 [==============================] - 3s 57ms/step - loss: 0.2784 - accuracy

In [23]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result

In [24]:
print(x_test[0])

[  69    7 1407   42 7891  581  206 2409   19 1560 4932    2   15  604
   66 2047   25   28 7892  344   13  271    5  358    4  121    8    3
    0    0    0    0    0    0    0    0    0    0    0    0]


In [25]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

23/23 [==============================] - 3s 70ms/step


c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.58      0.57      0.58       657
         _DT       0.93      0.88      0.91       335
         _LC       0.69      0.59      0.64       312
         _OG       0.70      0.56      0.62       481
         _PS       0.73      0.48      0.58       374
         _TI       0.86      0.82      0.84        66

   micro avg       0.71      0.61      0.65      2225
   macro avg       0.75      0.65      0.69      2225
weighted avg       0.71      0.61      0.65      2225

F1-score: 65.5%


In [26]:
print(pred_tags[0])
print(test_tags[0])

['B_DT', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['B_DT', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [27]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print("새로운 유형의 시퀀스 : ", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding="post", value=0, maxlen=max_len)
print("새로운 유형의 패딩된 시퀀스 : ", new_padded_seqs)
print(len(new_padded_seqs[0]))

새로운 유형의 시퀀스 :  [531, 307, 1476, 286, 1507, 6766, 1]
새로운 유형의 패딩된 시퀀스 :  [[ 531  307 1476  286 1507 6766    1    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
40


In [28]:
# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
print('p : ', p)
p = np.argmax(p, axis=-1)
print('index of each maximum elements of p : ', p)
print('length of p :', len(p[0]))# 예측된 NER 인덱스값 추출
# only 7 is real values, rest is padding!
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)

1/1 [==============================] - 0s 73ms/step
p :  [[[1.14090897e-08 4.71777275e-05 3.35414516e-05 9.99789417e-01
   1.26415704e-04 4.02266380e-07 3.03250022e-06 7.17077953e-10]
  [1.76694233e-08 9.99198020e-01 7.94239342e-04 5.22262144e-06
   2.07837229e-06 1.81124449e-07 2.30003820e-07 2.63181144e-09]
  [3.41471974e-07 9.99873281e-01 4.21222358e-05 3.34498072e-05
   9.97941606e-06 1.58340263e-05 2.49230634e-05 4.99451396e-08]
  [4.85833198e-07 2.72453268e-04 1.01488104e-05 2.05194403e-04
   1.56557025e-08 9.99207437e-01 2.05381759e-04 9.89314867e-05]
  [1.75834120e-05 2.93629663e-03 1.70796812e-02 9.77409363e-01
   2.02865092e-04 1.23806632e-04 2.20340630e-03 2.70485743e-05]
  [1.21113926e-03 5.00886142e-01 4.80825007e-01 5.79342153e-03
   4.10142005e-04 2.25596596e-03 4.72472515e-03 3.89338634e-03]
  [4.17524064e-03 5.28384626e-01 4.28981870e-01 8.61465838e-03
   1.21535745e-03 3.55494837e-03 7.58039393e-03 1.74929388e-02]
  [1.19524889e-01 1.48132503e-01 1.20998979e-01 1.2128

In [29]:
for w, pred in zip(new_sentence, p[0]):
    print(w, pred)
    print("{:10} {:5}".format(w, index_to_ner[pred]))
    print('-' * 20)

삼성전자 3
삼성전자       B_OG 
--------------------
출시 1
출시         O    
--------------------
스마트폰 1
스마트폰       O    
--------------------
오늘 5
오늘         B_DT 
--------------------
애플 3
애플         B_OG 
--------------------
도전장 1
도전장        O    
--------------------
내밀다. 1
내밀다.       O    
--------------------
